In [ ]:
# Data + Augmentation 
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


batch_size = 32

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform_train)

trainset, validationset = random_split(dataset, [40000, 10000])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, prefetch_factor=2, num_workers=8,pin_memory=True)
validationloader = torch.utils.data.DataLoader(validationset, batch_size=batch_size, shuffle=True, prefetch_factor=2, num_workers=8,pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, prefetch_factor=2, num_workers=8,pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
#Utility Functions
def compute_accuracy(net, dataloader):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    
    for data in dataloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        # calculate outputs by running images through the network
        outputs = net(images.view(images.size(0), -1))
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

def numel(m: torch.nn.Module, only_trainable: bool = False):
    """
    returns the total number of parameters used by `m` (only counting
    shared parameters once); if `only_trainable` is True, then only
    includes parameters with `requires_grad = True`
    """
    parameters = m.parameters()
    if only_trainable:
        parameters = list(p for p in parameters if p.requires_grad)
    unique = dict((p.data_ptr(), p) for p in parameters).values()
    return sum(p.numel() for p in unique)

In [ ]:
# Model Classes
class LinearBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearBlock, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.bn = nn.BatchNorm1d(output_dim)
    
    def forward(self, x):
        x = self.fc(x)
        x = F.relu(x)
        x = self.bn(x)
        
        return x
    
class LinearNet(nn.Module):
    def __init__(self, input_dim ,output_dim, layers, block = LinearBlock, num_classes=10):
        super(LinearNet, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layers = layers
        self.block = block
        self.feed = self.make_layers()
        

    def make_layers(self):
        network_layers = []
        network_layers.append(
            LinearBlock(self.input_dim, self.layers[0])
        )
        
        for i in range(len(self.layers) - 1):
            network_layers.append(
                self.block(self.layers[0], self.layers[i+1])
            )
            
        network_layers.append(
            nn.Linear(self.layers[0], self.output_dim)
        )
        

        return nn.Sequential(*network_layers)
    
    def forward(self, x):
        x = self.feed(x)
        return x

# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 32
        self.conv = conv3x3(3, 32)
        self.bn = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 128, layers[0])
        self.layer2 = self.make_layer(block, 256*2, layers[1], 2)
        self.layer3 = self.make_layer(block, 256*2, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(256*2, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [ ]:
# Initialize Network and Storage
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')

teacher = Net().to(device)
teacher.load_state_dict(torch.load("./Teacher260"))
layers = [256 for i in range(8)] + [512 for i in range(8)] + [1024 for i in range(12)]
student = LinearNet(32*32*3, 10, layers, block = ResLinearBlock).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(student.parameters())#, #momentum=0.9)
epochs_done=0
print(numel(student)/10**6)

train_accuracy = []
validation_accuracy = []

validation_loss = []
train_loss = []

In [ ]:
#Training Loop
#Training Loop
for epoch in range(300):  # loop over the dataset multiple times
    epochs_done += 1 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        
        inputs, _ = data
        inputs = inputs.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = student(inputs)
        with torch.no_grad():
            teacher_outputs = teacher(inputs)
            
        loss = criterion(outputs, teacher_outputs)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    #if epochs_done % 10 == 0 and epochs_done!=0 :
        #torch.save(student.state_dict(), f"./distillation2_linear_3_{epochs_done}")
        
        
    if epoch % 5 == 0:
        with torch.no_grad(): 
            v_loss = 0.0
            for j, data in enumerate(validationloader, 0):
                inputs, _ = data

                inputs = inputs.to(device)
                teacher_outputs = F.softmax(teacher(inputs))
                
                outputs = F.softmax(student(inputs))
                v_loss += criterion(outputs, teacher_outputs).item()
                
            validation_loss.append(v_loss/j)
            
            v_accuracy = compute_accuracy(student, validationloader)
            validation_accuracy.append(v_accuracy)
            
            train_loss.append(running_loss/i)
            t_accuracy = compute_accuracy(student, trainloader)
            train_accuracy.append(t_accuracy)
    
    print(f"Epoch: {epochs_done}, training loss: {running_loss/i}, validation loss: {v_loss/j}, validation accuracy: {v_accuracy}%")
                
print('Finished Training')

In [ ]:
#Plot Accuracy and Loss
import matplotlib.pyplot as plt

epochs_plot = [5*(i + 1) for i in range(epoch//5 + 1)]

plt.plot(epochs_plot, validation_loss)
plt.plot(epochs_plot, train_loss)
plt.title("Loss Plot")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Validation Loss", "Training Loss"])
plt.show()
#plt.savefig("linear_deep_distillation_300_loss.png", dpi=None, facecolor='w', edgecolor='w',
#        orientation='portrait', papertype=None, format=None,
#        transparent=False, bbox_inches=None, pad_inches=0.1,
#        frameon=None, metadata=None)

#plt.clf()

plt.plot(epochs_plot, validation_accuracy)
plt.plot(epochs_plot, train_accuracy)
plt.title("Accuracy Plot")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Validation Accuracy", "Training Accuracy"])
plt.show()
#plt.savefig("linear_deep_distillation_300_acc.png", dpi=None, facecolor='w', edgecolor='w',
#        orientation='portrait', papertype=None, format=None,
#        transparent=False, bbox_inches=None, pad_inches=0.1,
#        frameon=None, metadata=None)

#plt.clf()